In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [ ]:
!pip install matplotlib

In [6]:
import matplotlib.pyplot as plt

In [8]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root = './data',train = True, download = True,transform = transform)
testset = torchvision.datasets.CIFAR10(root = './data',train = False, download = True,transform = transform)

trainloader = torch.utils.data.DataLoader(trainset,batch_size = 64, shuffle=True, num_workers = 2)
testloader = torch.utils.data.DataLoader(testset,batch_size = 64, shuffle=True, num_workers = 2)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
# do I have cuda
torch.cuda.is_available()

True

In [10]:
# after making sure cuda is available, we need to create a cuda device object that represent particular nvidia GPU 
# if GPU is not available we put an else to avoid error , using cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
from torch.nn.modules.pooling import MaxPool2d
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
       nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3,stride=1,padding=1),  # (i-f+2p)/s  + 1  (32-3+2)/1  +1
       nn.MaxPool2d(kernel_size=2,stride=2), # (M-P)/s + 1  (32-2)/2 + 1 = 16
       nn.ReLU(),
       nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3,stride=1,padding=1), 
       nn.MaxPool2d(kernel_size=2,stride=2),   # (M-P)/s + 1  (16-2)/2 +1  = 8
       nn.ReLU()
    )
    self.fc = nn.Sequential(
       nn.Linear(in_features=32*8*8,out_features=64),
       nn.ReLU(),
       nn.Linear(in_features=64,out_features=10)  
    )

  def forward(self,x):
    x = self.conv(x)
    x = x.view(x.size(0),-1)
    x = self.fc(x)
    return x


# the only line is different from previous version with out CUDA is here, it was 'model = CNN()', but now :
model = CNN().to(device)
# we refere the convolutional network instance called model, to device object to use cuda instead of cpu 

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01)

In [13]:
import time
start_time = time.time()

loss_list = []
accuracy_list = []
for epoch in range(10):
    epoch_loss = 0
    for images, labels in trainloader:
        # image and labels also should be refered to device object using cuda
        # so we add these two lines
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    loss_list.append(epoch_loss / len(trainloader))

        
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in trainloader:
            # image and labels also should be refered to device object using cuda
            # so we add these two lines
            images = images.to(device)
            labels = labels.to(device)
            
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = correct / total
            accuracy_list.append(accuracy)

        print("Epoch [{}/30] Training Accuracy: {:.4f}".format(epoch + 1, correct / total),"Loss: {:.4f}".format(loss_list[-1]))

print("Training took {:.2f} seconds".format(time.time() - start_time))


# the training time process using GPU took less time, GPU is way faster
# 

Epoch [1/30] Training Accuracy: 0.3002 Loss: 2.1030
Epoch [2/30] Training Accuracy: 0.3673 Loss: 1.7227
Epoch [3/30] Training Accuracy: 0.4755 Loss: 1.5241
Epoch [4/30] Training Accuracy: 0.4723 Loss: 1.4137
Epoch [5/30] Training Accuracy: 0.5396 Loss: 1.3344
Epoch [6/30] Training Accuracy: 0.5258 Loss: 1.2677
Epoch [7/30] Training Accuracy: 0.5717 Loss: 1.2102
Epoch [8/30] Training Accuracy: 0.6022 Loss: 1.1596
Epoch [9/30] Training Accuracy: 0.5729 Loss: 1.1136
Epoch [10/30] Training Accuracy: 0.6142 Loss: 1.0757
Training took 207.45 seconds


In [15]:
with torch.no_grad():
  correct = 0 
  total = 0
  for images, labels in testloader:
    # image and labels also should be refered to device object using cuda
    # so we add these two lines
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy: {:.4f}'.format(correct/total))

Test Accuracy: 0.5970


In [17]:
# create a dictionary to save th etrained model
save_state = {'model':model.state_dict(), 'optimizer':optimizer.state_dict()}
torch.save(save_state, r'C:\Users\TE580354\OneDrive - TE Connectivity\51-1880822\focus_checker\code\CNN_GPU.pth')

In [18]:
# test the saved model if works, lets load the model and its optimizer
# u can use this part in a seperate notebook, withouttraining before, as the model is saved and we need just to load it
state = torch.load(r'C:\Users\TE580354\OneDrive - TE Connectivity\51-1880822\focus_checker\code\CNN_GPU.pth')
model.load_state_dict(state['model'])
optimizer.load_state_dict(state['optimizer'])

C:\Users\TE580354\AppData\Local\Temp\ipykernel_12596\4289054098.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(r'C:\Users\TE580354\OneDrive - TE Conn

In [19]:

with torch.no_grad():
  correct = 0 
  total = 0
  for images, labels in testloader:
    # to use GPU we add .to(device)
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy: {:.4f}'.format(correct/total))
  # so it worked

Test Accuracy: 0.5970
